In [8]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [9]:
df = pd.read_csv("diabetic_data.csv")
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


## Reproducing the 23-feature setup from Sah & Kanu (2023)
This section rebuilds the 23-feature subset described in the report, mirrors their ordinal encodings, and benchmarks logistic regression, decision tree, and random forest models to compare with the published figures.

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix
 )

AGE_INTERVALS = {
    "[0-10)": (0, 10),
    "[10-20)": (10, 20),
    "[20-30)": (20, 30),
    "[30-40)": (30, 40),
    "[40-50)": (40, 50),
    "[50-60)": (50, 60),
    "[60-70)": (60, 70),
    "[70-80)": (70, 80),
    "[80-90)": (80, 90),
    "[90-100)": (90, 100)
}

RACE_ORDER = ["Caucasian", "AfricanAmerican", "Hispanic", "Asian", "Other"]

MED_CHANGE_MAP = {"No": 0, "Steady": 0, "Up": 1, "Down": 1}
A1C_MAP = {"None": 0, "Norm": 1, ">7": 2, ">8": 3}
GLU_MAP = {"None": 0, "Norm": 1, ">200": 2, ">300": 3}
CHANGE_MAP = {"No": 0, "Ch": 1}
DIABETES_MED_MAP = {"No": 0, "Yes": 1}
GENDER_MAP = {"Female": 0, "Male": 1}
HOME_DISCHARGE = {1, 6}
TRANSFER_DISCHARGE = {2, 3, 4, 5, 8, 9, 10, 15, 16, 17, 22, 23, 24, 25, 26, 30}
STILL_PATIENT_DISCHARGE = {12}
NOT_AVAILABLE_DISCHARGE = {18}
EXCLUDE_DISCHARGE = {11, 13, 14, 19, 20, 21}
EMERGENCY_ADMISSION = {1, 2, 7}
ELECTIVE_ADMISSION = {3}
NEWBORN_ADMISSION = {4}
NOT_AVAILABLE_ADMISSION = {5, 6, 8}
EMERGENCY_SOURCE = {7}
REFERRAL_SOURCE = {1, 2, 3}
TRANSFER_SOURCE = {4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26}
NOT_AVAILABLE_SOURCE = {9, 17}

def generate_age_numeric(age_series, seed=42):
    rng = np.random.default_rng(seed)
    numeric_ages = []
    for label in age_series:
        if pd.isna(label):
            numeric_ages.append(np.nan)
            continue
        interval = AGE_INTERVALS.get(label)
        if interval is None:
            numeric_ages.append(np.nan)
            continue
        low, high = interval
        numeric_ages.append(int(rng.integers(low, high)))
    return numeric_ages

def bucket_admission_type(value):
    if pd.isna(value):
        return np.nan
    value = int(value)
    if value in EMERGENCY_ADMISSION:
        return 0
    if value in ELECTIVE_ADMISSION:
        return 1
    if value in NEWBORN_ADMISSION:
        return 2
    if value in NOT_AVAILABLE_ADMISSION:
        return 3
    return 4

def bucket_discharge(value):
    if pd.isna(value):
        return np.nan
    value = int(value)
    if value in HOME_DISCHARGE:
        return 0
    if value in TRANSFER_DISCHARGE:
        return 1
    if value in STILL_PATIENT_DISCHARGE:
        return 2
    if value == 7:  # Left AMA
        return 3
    if value in NOT_AVAILABLE_DISCHARGE:
        return 4
    return 5

def bucket_admission_source(value):
    if pd.isna(value):
        return np.nan
    value = int(value)
    if value in EMERGENCY_SOURCE:
        return 0
    if value in REFERRAL_SOURCE:
        return 1
    if value in TRANSFER_SOURCE:
        return 2
    if value in NOT_AVAILABLE_SOURCE:
        return 3
    return 4

def map_icd9_to_group(code):
    if pd.isna(code):
        return 0
    code_str = str(code)
    if code_str.startswith('V') or code_str.startswith('E') or code_str == '-1':
        return 0
    try:
        code_val = float(code_str)
    except ValueError:
        return 0
    if 390 <= code_val < 460 or code_val == 785:
        return 1  # Circulatory system
    if 460 <= code_val < 520 or code_val == 786:
        return 2  # Respiratory system
    if 520 <= code_val < 580 or code_val == 787:
        return 3  # Digestive system
    if 250 <= code_val < 251:
        return 4  # Diabetes
    if 800 <= code_val < 1000:
        return 5  # Injury
    if 710 <= code_val < 740:
        return 6  # Musculoskeletal
    if 580 <= code_val < 630:
        return 7  # Genitourinary
    if 140 <= code_val < 240:
        return 8  # Neoplasms
    if 240 <= code_val < 280:
        return 9  # Endocrine/metabolic
    if 680 <= code_val < 710:
        return 10  # Skin/subcutaneous
    if 320 <= code_val < 390:
        return 11  # Nervous system
    if 280 <= code_val < 320:
        return 12  # Blood diseases
    if 630 <= code_val < 680:
        return 13  # Pregnancy/childbirth
    if 740 <= code_val < 760:
        return 14  # Congenital anomalies
    if 760 <= code_val < 780:
        return 15  # Perinatal period
    if 780 <= code_val < 785:
        return 16  # Symptoms
    if 120 <= code_val < 140:
        return 17  # Infectious and parasitic
    return 18  # Residual/other

def encode_race(series):
    cleaned = series.fillna('Other').replace('?', 'Other')
    mapped = cleaned.apply(lambda x: RACE_ORDER.index(x) if x in RACE_ORDER else len(RACE_ORDER))
    return mapped

In [11]:
def prepare_paper_feature_set(df_raw):
    data = df_raw.copy()
    data = data.replace('?', np.nan)

    # Remove newborn admissions and excluded discharge categories
    if 'admission_type_id' in data.columns:
        data = data[~data['admission_type_id'].isin(NEWBORN_ADMISSION)]
    if 'discharge_disposition_id' in data.columns:
        data = data[~data['discharge_disposition_id'].isin(EXCLUDE_DISCHARGE)]

    # Drop identifiers and high-missing features that the report discards
    drop_cols = [
        'weight', 'payer_code', 'medical_specialty',
        'encounter_id', 'patient_nbr',
        'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin',
        'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone'
    ]
    data = data.drop(columns=[col for col in drop_cols if col in data.columns])

    # Basic imputations
    if 'race' in data.columns:
        data['race'] = data['race'].fillna(data['race'].mode()[0])
    diag_cols = ['diag_1', 'diag_2', 'diag_3']
    for col in diag_cols:
        if col in data.columns:
            data[col] = data[col].fillna('-1')

    # Grouped categorical encodings
    data['admission_type_group'] = data['admission_type_id'].apply(bucket_admission_type)
    data['discharge_disposition_group'] = data['discharge_disposition_id'].apply(bucket_discharge)
    data['admission_source_group'] = data['admission_source_id'].apply(bucket_admission_source)

    # Prior year utilization (sum of outpatient, emergency, inpatient visits)
    utilization_cols = ['number_outpatient', 'number_emergency', 'number_inpatient']
    for col in utilization_cols:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0)
        else:
            data[col] = 0
    data['prior_year_visits'] = data[utilization_cols].sum(axis=1)

    # Medication change flags
    for med in ['insulin', 'metformin', 'glipizide', 'glyburide']:
        if med in data.columns:
            data[f'{med}_change_flag'] = data[med].map(MED_CHANGE_MAP).fillna(0).astype(int)
        else:
            data[f'{med}_change_flag'] = 0

    # Ordinal encodings for lab results and medication program
    data['A1Cresult_ord'] = data['A1Cresult'].map(A1C_MAP).fillna(0).astype(int)
    data['max_glu_serum_ord'] = data['max_glu_serum'].map(GLU_MAP).fillna(0).astype(int)
    data['change_flag'] = data['change'].map(CHANGE_MAP).fillna(0).astype(int)
    data['diabetesMed_flag'] = data['diabetesMed'].map(DIABETES_MED_MAP).fillna(0).astype(int)

    # Demographic encodings
    data = data[data['gender'] != 'Unknown/Invalid']
    data['gender_flag'] = data['gender'].map(GENDER_MAP).fillna(0).astype(int)
    data['race_ord'] = encode_race(data['race']).astype(int)
    data['age_numeric'] = generate_age_numeric(data['age'])

    # Diagnosis groupings
    for col in diag_cols:
        data[f'{col}_group'] = data[col].apply(map_icd9_to_group).astype(int)

    # Target label
    data['readmitted_flag'] = (data['readmitted'] == '<30').astype(int)

    feature_cols = [
        'race_ord', 'gender_flag', 'age_numeric',
        'admission_type_group', 'discharge_disposition_group', 'admission_source_group',
        'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
        'number_diagnoses', 'prior_year_visits',
        'insulin_change_flag', 'metformin_change_flag', 'glipizide_change_flag', 'glyburide_change_flag',
        'max_glu_serum_ord', 'A1Cresult_ord', 'change_flag', 'diabetesMed_flag',
        'diag_1_group', 'diag_2_group', 'diag_3_group'
    ]

    features = data[feature_cols].apply(pd.to_numeric, errors='coerce')
    target = data['readmitted_flag']

    valid_mask = ~features.isna().any(axis=1)
    cleaned_features = features[valid_mask]
    cleaned_target = target[valid_mask]

    return cleaned_features, cleaned_target, data.loc[valid_mask, feature_cols + ['readmitted_flag']]

In [12]:
paper_X, paper_y, paper_dataset = prepare_paper_feature_set(df)
print(f"Samples after filtering: {len(paper_X):,}")
print(f"Positive rate (<30 days): {paper_y.mean():.3%}")
print(f"Feature count: {paper_X.shape[1]}")
paper_X.head()

Samples after filtering: 99,330
Positive rate (<30 days): 11.389%
Feature count: 23


,race_ord,gender_flag,age_numeric,admission_type_group,discharge_disposition_group,admission_source_group,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,metformin_change_flag,glipizide_change_flag,glyburide_change_flag,max_glu_serum_ord,A1Cresult_ord,change_flag,diabetesMed_flag,diag_1_group,diag_2_group,diag_3_group
0,0,0,0,3,1,1,1,41,0,1,...,0,0,0,0,0,0,0,4,0,0
1,0,0,17,0,0,0,3,59,0,18,...,0,0,0,0,0,1,1,9,4,9
2,1,0,26,0,0,0,2,11,5,13,...,0,0,0,0,0,0,1,13,4,0
3,0,1,34,0,0,0,2,44,1,16,...,0,0,0,0,0,1,1,18,4,1
4,0,1,44,0,0,0,1,51,0,8,...,0,0,0,0,0,1,1,8,8,4


In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    paper_X, paper_y, test_size=0.3, random_state=42, stratify=paper_y
 )

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, solver='lbfgs'),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
}

results = []
reports = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds, zero_division=0)
    recall = recall_score(y_test, preds, zero_division=0)
    f1 = f1_score(y_test, preds, zero_division=0)
    roc_auc = roc_auc_score(y_test, proba) if proba is not None else np.nan

    results.append({
        "Model": name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1,
        "ROC-AUC": roc_auc
    })
    reports[name] = classification_report(y_test, preds, output_dict=True, zero_division=0)

results_df = pd.DataFrame(results).set_index("Model").sort_values("ROC-AUC", ascending=False)
results_df

,Accuracy,Precision,Recall,F1,ROC-AUC
Model,,,,,
Random Forest,0.886171,0.545455,0.003536,0.007026,0.621729
Logistic Regression,0.885063,0.218182,0.003536,0.006959,0.620325
Decision Tree,0.788214,0.150156,0.184443,0.165543,0.525132


In [14]:
for name, report in reports.items():
    print("=" * 80)
    print(f"Classification report: {name}")
    print("=" * 80)
    display(pd.DataFrame(report).transpose())

print("=" * 80)
print("Confusion matrix (Random Forest)")
print("=" * 80)
rf_preds = models['Random Forest'].predict(X_test)
conf_matrix = pd.DataFrame(
    confusion_matrix(y_test, rf_preds),
    index=['Actual 0', 'Actual 1'],
    columns=['Predicted 0', 'Predicted 1']
 )
display(conf_matrix)

Classification report: Logistic Regression


,precision,recall,f1-score,support
0,0.886296,0.998372,0.939002,26405.000000
1,0.218182,0.003536,0.006959,3394.000000
accuracy,0.885063,0.885063,0.885063,0.885063
macro avg,0.552239,0.500954,0.472980,29799.000000
weighted avg,0.810201,0.885063,0.832845,29799.000000


Classification report: Decision Tree


,precision,recall,f1-score,support
0,0.892002,0.865821,0.878716,26405.000000
1,0.150156,0.184443,0.165543,3394.000000
accuracy,0.788214,0.788214,0.788214,0.788214
macro avg,0.521079,0.525132,0.522130,29799.000000
weighted avg,0.807508,0.788214,0.797488,29799.000000


Classification report: Random Forest


,precision,recall,f1-score,support
0,0.886422,0.999621,0.939625,26405.000000
1,0.545455,0.003536,0.007026,3394.000000
accuracy,0.886171,0.886171,0.886171,0.886171
macro avg,0.715938,0.501578,0.473325,29799.000000
weighted avg,0.847587,0.886171,0.833405,29799.000000


Confusion matrix (Random Forest)


,Predicted 0,Predicted 1
Actual 0,26395,10
Actual 1,3382,12


### Class-weighting and threshold tuning for Random Forest

In [15]:
rf_weighted = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced_subsample',
    min_samples_leaf=5
 )
rf_weighted.fit(X_train, y_train)
rf_weighted_proba = rf_weighted.predict_proba(X_test)[:, 1]
rf_weighted_preds = (rf_weighted_proba >= 0.5).astype(int)

weighted_metrics = {
    "Accuracy": accuracy_score(y_test, rf_weighted_preds),
    "Precision": precision_score(y_test, rf_weighted_preds, zero_division=0),
    "Recall": recall_score(y_test, rf_weighted_preds, zero_division=0),
    "F1": f1_score(y_test, rf_weighted_preds, zero_division=0),
    "ROC-AUC": roc_auc_score(y_test, rf_weighted_proba)
}
pd.Series(weighted_metrics)

Accuracy     0.876573
Precision    0.276730
Recall       0.051856
F1           0.087345
ROC-AUC      0.643674
dtype: float64

In [16]:
from sklearn.metrics import precision_recall_curve

In [17]:
precisions, recalls, thresholds = precision_recall_curve(y_test, rf_weighted_proba)
f1_scores = 2 * precisions * recalls / (precisions + recalls + 1e-12)
best_idx = np.nanargmax(f1_scores)
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
best_threshold, f1_scores[best_idx], recalls[best_idx], precisions[best_idx]

(0.3037172719469542,
 0.2633380884447115,
 0.5439010017678255,
 0.17372482589873894)

In [18]:
rf_weighted_preds_opt = (rf_weighted_proba >= best_threshold).astype(int)
threshold_metrics = {
    "Threshold": best_threshold,
    "Accuracy": accuracy_score(y_test, rf_weighted_preds_opt),
    "Precision": precision_score(y_test, rf_weighted_preds_opt, zero_division=0),
    "Recall": recall_score(y_test, rf_weighted_preds_opt, zero_division=0),
    "F1": f1_score(y_test, rf_weighted_preds_opt, zero_division=0),
    "ROC-AUC": roc_auc_score(y_test, rf_weighted_proba)
}
pd.Series(threshold_metrics)

Threshold    0.303717
Accuracy     0.653411
Precision    0.173725
Recall       0.543901
F1           0.263338
ROC-AUC      0.643674
dtype: float64

In [19]:
rf_weighted_cm = pd.DataFrame(
    confusion_matrix(y_test, rf_weighted_preds_opt),
    index=['Actual 0', 'Actual 1'],
    columns=['Predicted 0', 'Predicted 1']
 )
rf_weighted_cm

,Predicted 0,Predicted 1
Actual 0,17625,8780
Actual 1,1548,1846


In [20]:
comparison_df = results_df.copy()
comparison_df.loc['Random Forest (weighted, 0.5)'] = weighted_metrics
comparison_df.loc['Random Forest (weighted, tuned)'] = {k: threshold_metrics.get(k, np.nan) for k in ['Accuracy','Precision','Recall','F1','ROC-AUC']}
comparison_df

,Accuracy,Precision,Recall,F1,ROC-AUC
Model,,,,,
Random Forest,0.886171,0.545455,0.003536,0.007026,0.621729
Logistic Regression,0.885063,0.218182,0.003536,0.006959,0.620325
Decision Tree,0.788214,0.150156,0.184443,0.165543,0.525132
"Random Forest (weighted, 0.5)",0.876573,0.276730,0.051856,0.087345,0.643674
"Random Forest (weighted, tuned)",0.653411,0.173725,0.543901,0.263338,0.643674
